In [1]:
from pyspark.sql import SparkSession

In [3]:
spark = (SparkSession.builder
      .master('local[1]')
      .appName('SparkApp')
      .getOrCreate())

In [5]:
rdd2 = spark.sparkContext.textFile("data/test.txt")

In [8]:
rdd2.count()

8

In [9]:
rdd2.first()

'536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2.55,17850,United Kingdom'

In [10]:
rdd2.take(10)

['536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2.55,17850,United Kingdom',
 '536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,17850,United Kingdom',
 '536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 8:26,2.75,17850,United Kingdom',
 '536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 8:26,3.39,17850,United Kingdom',
 '536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,17850,United Kingdom',
 '536365,22752,SET 7 BABUSHKA NESTING BOXES,2,12/1/10 8:26,7.65,17850,United Kingdom',
 '536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,12/1/10 8:26,4.25,17850,United Kingdom',
 '536366,22633,HAND WARMER UNION JACK,6,12/1/10 8:28,1.85,17850,United Kingdom']

In [11]:
# Create DataFrame
data = [('James','','Smith','1991-04-01','M',3000),
        ('Michael','Rose','','2000-05-19','M',4000),
        ('Robert','','Williams','1978-09-05','M',4000),
        ('Maria','Anne','Jones','1967-12-01','F',4000),
        ('Jen','Mary','Brown','1980-02-17','F',-1)
        ]

In [12]:
columns = ["firstname","middlename","lastname","dob","gender","salary"]

In [13]:
df = spark.createDataFrame(data, columns)

In [16]:
df.show(5, 0)

+---------+----------+--------+----------+------+------+
|firstname|middlename|lastname|dob       |gender|salary|
+---------+----------+--------+----------+------+------+
|James    |          |Smith   |1991-04-01|M     |3000  |
|Michael  |Rose      |        |2000-05-19|M     |4000  |
|Robert   |          |Williams|1978-09-05|M     |4000  |
|Maria    |Anne      |Jones   |1967-12-01|F     |4000  |
|Jen      |Mary      |Brown   |1980-02-17|F     |-1    |
+---------+----------+--------+----------+------+------+



In [26]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

dataDF = [(('James','','Smith'),'1991-04-01','M',3000),
          (('Michael','Rose',''),'2000-05-19','M',4000),
          (('Robert','','Williams'),'1978-09-05','M',4000),
          (('Maria','Anne','Jones'),'1967-12-01','F',4000),
          (('Jen','Mary','Brown'),'1980-02-17','F',-1)
          ]

schema = StructType([
    StructField('name', StructType([
        StructField('firstname', StringType(), True),
        StructField('middlename', StringType(), True),
        StructField('lastname', StringType(), True),
    ])),
    StructField('dob', StringType(), True),
    StructField('gender', StringType(), True),
    StructField('salary', IntegerType(), True)
])

In [27]:
schema

StructType([StructField('name', StructType([StructField('firstname', StringType(), True), StructField('middlename', StringType(), True), StructField('lastname', StringType(), True)]), True), StructField('dob', StringType(), True), StructField('gender', StringType(), True), StructField('salary', IntegerType(), True)])

In [28]:
df = spark.createDataFrame(data=dataDF, schema=schema)

In [30]:
df.printSchema()

root
 |-- name: struct (nullable = true)
 |    |-- firstname: string (nullable = true)
 |    |-- middlename: string (nullable = true)
 |    |-- lastname: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)



In [33]:
df = df.withColumnRenamed("dob", "birthdate")

In [34]:
df.show()

+--------------------+----------+------+------+
|                name| birthdate|gender|salary|
+--------------------+----------+------+------+
|    {James, , Smith}|1991-04-01|     M|  3000|
|   {Michael, Rose, }|2000-05-19|     M|  4000|
|{Robert, , Williams}|1978-09-05|     M|  4000|
|{Maria, Anne, Jones}|1967-12-01|     F|  4000|
|  {Jen, Mary, Brown}|1980-02-17|     F|    -1|
+--------------------+----------+------+------+



In [36]:
df = df.withColumnRenamed("birthdate","DateOfBirth").withColumnRenamed("salary","salary_usd")
df.printSchema()

root
 |-- name: struct (nullable = true)
 |    |-- firstname: string (nullable = true)
 |    |-- middlename: string (nullable = true)
 |    |-- lastname: string (nullable = true)
 |-- DateOfBirth: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary_usd: integer (nullable = true)



In [37]:
schema2 = StructType([
    StructField("fname", StringType()),
    StructField("middlename", StringType()),
    StructField("lname", StringType()),
])

In [38]:
from pyspark.sql.functions import col

In [39]:
df.select(col("name").cast(schema2),
          col("DateOfBirth"), col("gender"), col("salary_usd")).printSchema()

root
 |-- name: struct (nullable = true)
 |    |-- fname: string (nullable = true)
 |    |-- middlename: string (nullable = true)
 |    |-- lname: string (nullable = true)
 |-- DateOfBirth: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary_usd: integer (nullable = true)



In [41]:
from pyspark.sql.functions import *

df.select(col("name.firstname").alias("fname"),
          col("name.middlename").alias("mname"),
          col("name.lastname").alias("lname"),
          col("dob"),col("gender"),col("salary")).printSchema()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `dob` cannot be resolved. Did you mean one of the following? [`name`, `gender`, `salary_usd`, `DateOfBirth`].;
'Project [name#87.firstname AS fname#165, name#87.middlename AS mname#166, name#87.lastname AS lname#167, 'dob, gender#89, 'salary]
+- Project [name#87, DateOfBirth#139, gender#89, salary#90 AS salary_usd#152]
   +- Project [name#87, birthdate#117 AS DateOfBirth#139, gender#89, salary#90]
      +- Project [name#87, dob#88 AS birthdate#117, gender#89, salary#90]
         +- LogicalRDD [name#87, dob#88, gender#89, salary#90], false
